$$
% Surprised you caught this, good eye!  Nothing exciting to see here though - just some custom commands that make typing math a lot easier
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
$$

# 11-860 Lab 2 - Implementing Single Qubit Gates
by Thomas Cantalapiedra


## Introduction
As we have learned in class, there are various gates that we can use to alter the state of a single qubit.  In this lab, we will play around with a few of them by coding gates in TKET and showing how these gates affect the phasor on the Bloch Sphere.

## Goal
The goal of this lab is to build a mental image about what these gates are doing.  This will be important for understanding how states are manipulated in the algorithms we will cover.

## Important Terminology
**Amplitude**:  The amplitude on $\ket{\omega}$ is a complex value telling us the probability of measuring the state to $\ket{\omega}.$  For example, if $$\ket{\psi} = \alpha \ket{0} + \beta \ket{1}$$ then the amplitude on $\ket{0}$ is $\alpha$.  

**Phase**:  You can think of phase as the argument we pass to the complex part of the amplitude.  For example, if the state is $$\ket{\psi} = \alpha \ket{0} + \beta \ket{1}$$ then the amplitudes can be written in polar form as $$\alpha = |\alpha | e^{i\phi_\alpha} \text{ and } \beta = |\beta | e^{i\phi_\beta}$$  The phase of $\ket{0}$ is $\phi_\alpha$ and the phase of $\ket{1}$ is $\phi_\beta.$

**Relative Phase**:  The relative phase is the difference in phase between amplitudes of a quantum superposition.  For example, if the state is $$\ket{\psi} = \alpha \ket{0} + \beta \ket{1}$$ then the amplitudes can be written in polar form as $$\alpha = |\alpha | e^{i\phi_\alpha} \text{ and } \beta = |\beta | e^{i\phi_\beta}$$  The relative phase of this state is $\Delta \phi = \phi_\beta - \phi_\alpha$.

In the Bloch Sphere, the azimuth angle ($\phi$) actually shows us the relative phase (disregarding a global phase factor that we will not cover).

In [ ]:
!pip install pytket
!pip install pytket-quantinuum[pecos]
import numpy as np
import pickle
import pytket as tket
import pytket.qasm as qasm
from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumAPIOffline
api_offline = QuantinuumAPIOffline()
backend = QuantinuumBackend(device_name="H1-1LE", api_handler = api_offline)
from pytket.circuit.display import render_circuit_jupyter


## NOT (Pauli-X) ($\sigma_x$)

The NOT gate (more formally known as the Pauli-X gate) is straightforward.  It simply flips the amplitudes on whatever state we pass to it.  On the block sphere, it rotates the phasor by $180°$ ($\pi$ radians) around the X-axis on the Bloch sphere. Its transition matrix is $$\begin{array}{c c}
& \begin{array}{c c} \ket{0} & \ket{1} \\ \end{array} \\
\begin{array}{c c} \ket{0} \\ \ket{1} \end{array} &
\left[
\begin{array}{c c}
0 & 1 \\
1 & 0
\end{array}
\right]
\end{array} $$

Note that in a transition matrix, the **columns** correspond to the **input states** and the *rows* correspond to the *output states*.  Why is this?

In [ ]:
X_circuit = tket.Circuit(1) # Create a new quantum circuit with 1 qubit
print("Apply NOT to the first bit")
X_circuit.X(0)
render_circuit_jupyter(X_circuit)

print("Apply NOT again")
X_circuit.X(0)
render_circuit_jupyter(X_circuit)

## Hadamard

The Hadamard gate ($H$ gate) is one of the most important gates in quantum computing.  If applied uniformly to all qubits, it puts our state into a superposition of all $2^N$ classical states where each one has an equal probability of being measured out.  This is called the "Uniform Superposition" and is used in almost every quantum algorithm because it allows us to test all possible inputs at once, with no initial bias towards any one input.  Its transition matrix is $$\begin{array}{c c}
& \begin{array}{c c} \ket{0} & \ket{1} \\ \end{array} \\
\begin{array}{c c} \ket{0} \\ \ket{1} \end{array} &
\left[
\begin{array}{c c}
\sqrt{\frac{1}{2}} & \sqrt{\frac{1}{2}}\\
\sqrt{\frac{1}{2}} & -\sqrt{\frac{1}{2}}
\end{array}
\right]
\end{array} $$

It is important to note that the Hadamard applied to $\ket{0}$ yields $$\sqrt{\frac{1}{2}}\ket{0} + \sqrt{\frac{1}{2}}\ket{1} = \ket{+}$$ and applied to $\ket{1}$ yields $$\sqrt{\frac{1}{2}}\ket{0} - \sqrt{\frac{1}{2}}\ket{1} = \ket{-}.$$

Indeed, if we wish to transform (or measure) into the sign basis, we must apply the Hadamard operation to our state.

In [ ]:
H_circuit = tket.Circuit(1,1)
print("Apply Hadamard to the first bit")
H_circuit.H(0)
H_circuit.Measure(0)
render_circuit_jupyter(H_circuit)

compiled = backend.get_compiled_circuit(H_circuit, optimisation_level=0)
qjob = backend.run_circuit(compiled, n_shots=100)
print(qjob.get_shots())

print("Apply Hadamard again")
H_circuit.H(0)
render_circuit_jupyter(H_circuit)

## Pauli-Z ($\sigma_z$)
The Pauli-Z gate ($Z$ gate for short) is an important gate for introducing phase flips to the state.  You can think of this as "if 1 then negate" - meaning that the amplitude on $\ket{0}$ is unchanged but any amplitude on $\ket{1}$ is negated.  This is a phase flip because it rotates the phasor around the Z-axis of the Bloch sphere by $180°$ ($\pi$ radians).  Its transition matrix is $$\begin{array}{c c}
& \begin{array}{c c} \ket{0} & \ket{1} \\ \end{array} \\
\begin{array}{c c} \ket{0} \\ \ket{1} \end{array} &
\left[
\begin{array}{c c}
1 & 0 \\
0 & -1
\end{array}
\right]
\end{array} $$

In [ ]:
# You will implement the Pauli-Z gate by looking through the TKET documentation
# https://docs.quantinuum.com/tket/api-docs/circuit_class.html

## Phase Gate ($S$ Gate)
The $S$ gate allows us to shift the phase of $\ket{1}$ by $90°$ ($\frac{\pi}{2}$ radians) about the Z-axis. Two of these operations is equivalent to one $Z$ gate.  Its transition matrix is $$\begin{array}{c c}
& \begin{array}{c c} \ket{0} & \ket{1} \\ \end{array} \\
\begin{array}{c c} \ket{0} \\ \ket{1} \end{array} &
\left[
\begin{array}{c c}
1 & 0 \\
0 & i
\end{array}
\right]
\end{array} $$
Exercise: Prove that this is equal to
$$\begin{array}{c c}
& \begin{array}{c c} \ket{0} & \ket{1} \\ \end{array} \\
\begin{array}{c c} \ket{0} \\ \ket{1} \end{array} &
\left[
\begin{array}{c c}
1 & 0 \\
0 & e^{i\pi/2}
\end{array}
\right]
\end{array} $$

In [ ]:
# You will implement the S gate by looking through the TKET documentation
# https://docs.quantinuum.com/tket/api-docs/circuit_class.html

## $T$ Gate
The $T$ gate allows us to shift the phase of $\ket{1}$ by $45°$ ($\frac{\pi}{4}$ radians) about the Z-axis. Four of these operations is equivalent to one $Z$ gate, and two of them is equivalent to one $S$ gate.  Its transition matrix is $$\begin{array}{c c}
& \begin{array}{c c} \ket{0} & \ket{1} \\ \end{array} \\
\begin{array}{c c} \ket{0} \\ \ket{1} \end{array} &
\left[
\begin{array}{c c}
1 & 0 \\
0 & e^{i \pi/4}
\end{array}
\right]
\end{array} $$

In [ ]:
# You will implement the T gate by looking through the TKET documentation
# https://docs.quantinuum.com/tket/api-docs/circuit_class.html

## Pauli-Y Gate ($\sigma_y$)
The Pauli-Y gate combines the concepts of bit flips (Pauli-X) and phase flip (Pauli-Z).  It rotates the phasor $180°$ ($\pi$ radians) about the Y-axis.  Its transition matrix is $$\begin{array}{c c}
& \begin{array}{c c} \ket{0} & \ket{1} \\ \end{array} \\
\begin{array}{c c} \ket{0} \\ \ket{1} \end{array} &
\left[
\begin{array}{c c}
0 & -i \\
i & 0
\end{array}
\right]
\end{array} $$
The bit flip part is obvious, so let's look at the phase. When we apply it to $\ket{0}$, we flip the amplitudes apply a phase shift of $i$ to the new amplitude of $\ket{1}$.  When we apply it to $\ket{1}$, we apply a phase shift of $-i$ to reverse this.

It may not seem intuitive how this combination of operations results in a phase shift of $\pi$, but rest assured that this is indeed how we rotate a phasor $180°$ about the Y-axis.

In [ ]:
# You will implement the Pauli-Y gate by looking through the TKET documentation
# https://docs.quantinuum.com/tket/api-docs/circuit_class.html

## Hermitian ($†$)
The Hermitian (also known as the "dagger") is the result of transposing a matrix and taking the complex conjugate of its elements.  As a reminder, the complex conjugate of $a + bi$ is $a - bi$.

Hermitians are important because they are the inverse of our unitary transformations.

## $S^\dagger$

$S^\dagger$ is the inverse of the $S$ gate.  Its transition matrix is $$\begin{array}{c c}
& \begin{array}{c c} \ket{0} & \ket{1} \\ \end{array} \\
\begin{array}{c c} \ket{0} \\ \ket{1} \end{array} &
\left[
\begin{array}{c c}
1 & 0 \\
0 & -i
\end{array}
\right]
\end{array} $$

Exercise: Show that $SS^\dagger = I$

## Now You Know All The Basic Single Qubit Gates!

Some terminology to know is that the Pauli, S, and Hadamard gates ($\{\sigma_x, \sigma_y, \sigma_z, S, H\}$) are referred to as Clifford gates and they have some cool properties.

Exercise: What is $H \sigma_z H^\dagger$?  What is $H \sigma_x H^\dagger$?

Exercise: What is $S \sigma_z S^\dagger$?  What is $S \sigma_x S^\dagger$?